# Ekstrakcja szkieletu (pose estimation)

Do realizacji projektu wybrano framework **MMPose** (https://github.com/open-mmlab/mmpose) z ekosystemu OpenMMLab.

## Wczytanie sekwencji wideo / klatek

### Wstępne ustawienia (detektor, pose estimator)

- **Detektor**: Faster R-CNN
- **Pose estimator**: HRNet-32

In [ ]:
# ustawienia detektora
detector_cfg = "../mmpose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py"
detector_chkp = "../mmpose/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth"

# ustawienia pose estimatora
pose_cfg = "../mmpose/model_configs/td-hm_hrnet-w32_8xb64-210e_coco-256x192.py"
pose_chkp = "../mmpose/checkpoints/td-hm_hrnet-w32_8xb64-210e_coco-256x192-81c58e40_20220909.pth"

# plik ze skryptem
script = "../mmpose/demo/topdown_demo_with_mmdet.py"

### Wczytanie i przetworzenie wszystkich sekwencji klatek na pliki wideo (mp4)

In [9]:
import os
from tqdm import tqdm # pasek postępu

# ścieżka do sekwencji klatek (?)
frames_folder = f"../UR_fall_det/frames"
# folder wyjściowy (do zapisu wyników)
output_folder = "../UR_fall_det/videos"

# tworzenie folderu wyjściowego jeśli nie istnieje
os.makedirs(output_folder, exist_ok=True)

# filtrowanie aby mieć katalogi z klatkami
folders = [f for f in os.listdir(frames_folder) if os.path.isdir(os.path.join(frames_folder, f))]
# sortowanie
folders = sorted(folders)

# pętla przetwarzająca
for folder_name in tqdm(folders, desc="Konwertowanie"):
    
    input_folder_path = os.path.join(frames_folder, folder_name)
    output_video_path = os.path.join(output_folder, f"{folder_name}-vis.mp4")

    # Wzorzec: "fall-01-cam0-rgb-%03d.png" (%03d oznacza 3 cyfry: 001, 002...)
    file_pattern = f'{folder_name}-%03d.png' 

    input_pattern_path = os.path.join(input_folder_path, file_pattern)

    # FFMPEG
    cmd = (
        f'ffmpeg -y -framerate 30 '
        f'-i "{input_pattern_path}" '
        f'-c:v libx264 -pix_fmt yuv420p '
        f'"{output_video_path}"'
    )

    exit_code = os.system(cmd)

    if exit_code != 0:
        print(f"\nBŁĄD przy folderze {folder_name}.")

print("\nZakończono konwersję sekwencji klatek do wideo.")

Konwertowanie: 100%|██████████| 70/70 [04:08<00:00,  3.54s/it]


Zakończono konwersję sekwencji klatek do wideo.


### Użycie detektora MMPose do wyznaczenia keypointów i bounding boxów dla każdej klatki wideo

In [12]:
import os
import sys
from tqdm import tqdm

# ścieżka do filmów
input_videos_folder = "../UR_fall_det/videos"
# folder wyjściowy
output_videos_folder = "../results"

# filtrowanie aby mieć pliki .mp4
videos = [v for v in os.listdir(input_videos_folder) if v.endswith('.mp4')]
videos = sorted(videos)

print(f"Znaleziono {len(videos)} plików wideo do przetworzenia.")

print("\nPrzechodzę do detekcji keypointów w MMPose...")

# pętla po plikach
for video_file in tqdm(videos, desc="Przetwarzanie wideo"):

    input_video_path = os.path.join(input_videos_folder, video_file)

    video_name_only = os.path.splitext(video_file)[0]
    output_folder_path = os.path.join(output_videos_folder, video_name_only)

    command = (
        f'{sys.executable} '        # Odpowiednik 'python' (gwarantuje użycie tego samego środowiska)
        f'{script} '                # skrypt
        f'{detector_cfg} '          # plik config detektora
        f'{detector_chkp} '         # checkpoint detektora
        f'{pose_cfg} '              # plik config pose estimatora
        f'{pose_chkp} '             # checkpoint pose estimatora
        f'--input "{input_video_path}" '               # wideo wejściowe
        f'--output-root "{output_folder_path}" '       # folder wynikowy
        f'--save-predictions '                         # aby zapisać predykcje w formacie JSON
        f'--draw-bbox '                                # aby rysować bounding boxy
        f'--device cuda:0'                             # wymuszenie GPU (można zakomentować)
    )

    process = os.system(command)

    if process != 0:
        print(f"\nBŁĄD przy wideo {video_name_only}.")

print("\nZakończono detekcję wideo z sukcesem.")

Znaleziono 70 plików wideo do przetworzenia.

Przechodzę do detekcji keypointów w MMPose...


Przetwarzanie wideo:  33%|███▎      | 23/70 [34:16<59:52, 76.44s/it]   


BŁĄD przy wideo adl-23-cam0-rgb-vis.


Przetwarzanie wideo: 100%|██████████| 70/70 [1:27:17<00:00, 74.83s/it]   


Zakończono detekcję wideo z sukcesem.


## Wizualizacja

In [4]:
import random
import os
from IPython.display import Video
import cv2

# ścieżka do filmów z wyznaczonymi keypointami
results_video_path = "../results/"

# filtrowanie aby mieć katalogi z wideo
results_videos = [r for r in os.listdir(results_video_path) if os.path.isdir(os.path.join(results_video_path, r))]
# sortowanie
results_videos = sorted(results_videos)

# wybieramy losowe wideo folderu
# można też po prostu podać ścieżke do konkretnego wideo
selected_video = random.choice(results_videos) # <-- można zamienić na np "fall-01-cam0-rgb-vis.mp4"

# pełna ścieżka
full_video_path = os.path.join(results_video_path, selected_video)
full_video_path = os.path.join(full_video_path, f"{selected_video}.mp4")

# print(f"Otwieram w systemowym odtwarzaczu: {selected_video}")
# # Działa tylko na Windows:
# os.startfile(full_video_path)

cap = cv2.VideoCapture(full_video_path)

if not cap.isOpened():
    print("Błąd: Nie można otworzyć pliku wideo.")
else:
    print(f"Odtwarzam wideo za pomocą OpenCV: {selected_video}")
    print("Naciśnij 'q' na klawiaturze (w oknie wideo), aby zamknąć.")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Wyświetlanie klatki w oknie "Podgląd"
            cv2.imshow(f'{selected_video}', frame)
            
            # Czekaj 25ms (ok. 40 FPS) i sprawdź czy wciśnięto 'q'
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

Odtwarzam wideo za pomocą OpenCV: fall-17-cam0-rgb-vis
Naciśnij 'q' na klawiaturze (w oknie wideo), aby zamknąć.
